In [1]:
!pip install requests
!pip install pandas
!pip install beautifulsoup4
!pip install mysql-connector-python
!pip install SQLAlchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 KB 885.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 KB 686.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 20.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 34.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 KB 1.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 17.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
resp = requests.get('https://meduza.io/')
resp

<Response [200]>

In [4]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [5]:
def first_or_none(elements):
    if len(elements):
        return elements[0].text
    else:
        return None


data = []
containers = soup.find_all("div", {"class": "Feed-container"})
for container in containers:
    children = container.findChildren('div', recursive=False)
    for child in children:
        blocks = child.find_all("h2", {"class":"BlockTitle-root"})
        for block in blocks:
            title = first_or_none(block.find_all("span", {"class":"BlockTitle-first"}))
            subtitle = first_or_none(block.find_all("span", {"class":"BlockTitle-second"}))
            if title: 
                row = {
                    'title': title,
                    'subtitle': subtitle
                }
                data.append(row)        

In [6]:
df = pd.DataFrame.from_dict(data)

Saving data to database

In [7]:
from sqlalchemy import create_engine, event
DATABSE_URI='mysql+mysqlconnector://{user}:{password}@{server}/{database}'.format(user='user', password='password', server='db:3306', database='test_db')
engine = create_engine(
                DATABSE_URI
            )
df.to_sql(con=engine, name='data', if_exists='replace')

36

Selecting data from database:

In [8]:
conn = engine.connect()
with conn:
    result = conn.execute("SELECT * FROM data;")
    for x in result:
        print(x)

(0, 'Полиция Украины задержала подозреваемых в\xa0подготовке беспорядков. Их\xa0собирались устроить в\xa0Киеве и\xa0регионах, расположенных в\xa0часе езды от\xa0России', None)
(1, 'В России новый суточный рекорд заражений ковидом\xa0— 124\u202f070 случаев. В\xa0Москве вводят дополнительные койки в\xa0детских стационарах', None)
(2, 'Danone предупредила о\xa0росте цен на\xa0свою продукцию в\xa0России на\xa010%', None)
(3, '«Мемориал» обжаловал решение Верховного суда РФ\xa0о\xa0ликвидации', None)
(4, 'В Подмосковье военнослужащий «непроизвольным выстрелом» убил сослуживца', None)
(5, '«Коммерсант»: МВД осталось недовольно работой серверов на\xa0отечественных процессорах «Эльбрус»', None)
(6, 'Spotify поменял правила модерации из-за бойкота Нила Янга и\xa0Джони Митчелл', 'Сервис запретил отрицать коронавирус и\xa0призывать к\xa0участию в\xa0ковид-вечеринках')
(7, 'Вышел трейлер долгожданного сериала по\xa0Halo', 'В нем мастер Чиф дерется с\xa0инопланетянами\xa0(не\xa0может быть!)')
(8, '